![Logo TUBAF](https://tu-freiberg.de/sites/default/files/media/freiberger-alumni-netzwerk-6127/wbm_orig_rgb_0.jpg)

Exercise material of the MSc-level course **Advanced Soil Mechanics**.
Held at Technische Universität Bergakademie Freiberg.

Comments to:

*Prof. Dr. Thomas Nagel  
Chair of Soil Mechanics and Foundation Engineering  
Geotechnical Institute  
Technische Universität Bergakademie Freiberg.*

https://tu-freiberg.de/en/fakultaet3/gt/soilmechanics


# Von Spannungen, Verzerrungen und Verschiebungen zu Setzungen -- indirekte Setzungsberechnungen

Ein weiteres Verfahren, dass sich insbesondere bei mit der Tiefe varrierenden Baugrundeigenschaften oder bei Vorhandensein von Spannungsmessungen oder -prognosen, die nicht aus den bisher behandelten geschlossenen Lösungen resultieren, eignet, sind die indirekten Setzungsberechnungen.

In [1]:
#HIDDEN
import numpy as np #numerical methods
import matplotlib.pyplot as plt #plotting
import seaborn as sns
import sympy as sp
from ipywidgets import widgets
from ipywidgets import interact
from ipywidgets import interactive
sp.init_printing(use_latex='mathjax') #makes sympy output look nice

#Some plot settings
import plot_functions.plot_settings
%run plot_functions/settlements_plots.ipynb

Dafür wird der Baugrund in Schichten eingeteilt. Schichtgrenzen können bspw.

* geologisch vorgegeben sein;
* aus spannungsabhängigen Steifigkeitsänderungen resultieren;
* veränderliche Lagerungsdichten oder Konsistenzen widerspiegeln;
* durch die Berücksichtigung von Grundwasserständen (Wichte unter Auftrieb) notwendig werden.

Für jede Schicht wird mit Bezug auf ihre mittlere Tiefe

* die vertikale Überlagerungseffektivspannung vor Beginn der Baumaßnahme bestimmt;
* die effektive Vertikalspannungseröhung infolge der Baumaßnahme ermittelt (bspw. über die Methode der $i$ Beiwerte);
* der zur Ausgangsspannung und zum Spannungszuwachs gehörige Streifemodul als Sekantenmodul aus Druck-Setzungskurven bestimmt und daraus der Rechenmodul abgeleitet. 

Mit diesen Informationen kann für jede Schicht die Zusammendrückung (Teilsetzungsbeitrag) mittels

$$
    \Delta s_i = \frac{\Delta \sigma'_i}{E^*_i(\sigma'_{0,i},\Delta \sigma'_i)} h_i
$$

ermittelt werden.

Die Gesamtsetzung ergibt sich durch Summation über alle $n$ Schichten:

$$
    s = \sum \limits_{i=1}^n \Delta s_i
$$